**ML Pipeline with ZenMl**

In [1]:
#!pip install "zenml[server]"
#!zenml integration install sklearn -y
#!pip install pyparsing==2.4.2

In [2]:
NGROK_TOKEN="2Zqveb0pxvljK9KSCSvOwYD9DTB_NsSvTGvYSksPZtwGEMZZ"

In [3]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**ZenML Setup**

In [4]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠴ Initializing ZenML repository at /content.
ZenML repository initialized at /content.
⠴ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, pl

**Example Experimentation ML Code**

In [5]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None:
    """Train and test a Scikit-learn SVC classifier on digits"""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")


train_test()

Test accuracy: 0.9583333333333334


**Turning experiments into ML pipelines with ZenML**

In [6]:
from zenml.steps import step, Output


@step
def importer() -> Output(
    X_train=np.ndarray,
    X_test=np.ndarray,
    y_train=np.ndarray,
    y_test=np.ndarray,
):
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier."""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc

The @step decorator that you used to define your importer step is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
The @step decorator that you used to define your svc_trainer step is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your steps to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty
The @step decorator that you used to define your evaluator step is deprecated. Check out the 0.40.0 migration guide for mo

In [7]:
from zenml.pipelines import pipeline


@pipeline
def digits_pipeline(importer, trainer, evaluator):
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

The @pipeline decorator that you used to define your digits_pipeline pipeline is deprecated. Check out the 0.40.0 migration guide for more information on how to migrate your pipelines to the new syntax: https://docs.zenml.io/reference/migration-guide/migration-zero-forty.html


In [8]:
digits_svc_pipeline = digits_pipeline(
    importer=importer(), trainer=svc_trainer(), evaluator=evaluator()
)
digits_svc_pipeline.run(unlisted=True)

Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Initiating a new run for the pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using the Output class to define the outputs of your steps is deprecated. You should instead use the standard Python way of type annotating your functions. Check out our documentation https://docs.zenml.io/user-guide/advanced-guide/pipelining-features/configure-steps-pipelines#step-output-names for more information on how to assign custom names to your step outputs.
Using cached version of importer.
Using the Output class to define the outputs of your steps i

In [9]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}

    else:
        !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-2bebd5e1-a709-4bc6-946c-ddfc6caf1186


Opening tunnel named: http-8237-2bebd5e1-a709-4bc6-946c-ddfc6caf1186


INFO:pyngrok.process.ngrok:t=2023-12-21T17:42:59+0000 lvl=info msg="no configuration paths supplied"


t=2023-12-21T17:42:59+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2023-12-21T17:42:59+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2023-12-21T17:46:23+0000 lvl=info msg="join connections" obj=join id=d9337044d6e9 l=127.0.0.1:8237 r=203.163.245.140:49790
Got token from cookie
INFO:     203.163.245.140:0 - "GET /api/v1/workspaces/default/components?scope_type=orchestrator&sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2023-12-21T17:46:29+0000 lvl=info msg="join connections" obj=join id=a4343a2776fe l=127.0.0.1:8237 r=203.163.245.140:49790


t=2023-12-21T17:46:29+0000 lvl=info msg="join connections" obj=join id=a4343a2776fe l=127.0.0.1:8237 r=203.163.245.140:49790
Got token from cookie
INFO:     203.163.245.140:0 - "GET /api/v1/workspaces/default/components?scope_type=orchestrator&sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2023-12-21T17:46:45+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2023-12-21T17:46:45+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:     Finished server process [71973]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError

